In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials

def update_google_sheet(service, spreadsheet_id, sheet_name, titles, descriptions, scores):
    """Update the sheet with titles, descriptions, and scores in separate columns."""
    # Update titles in column B
    title_range = f"{sheet_name}!B2:B"
    title_body = {'values': [[title] for title in titles]}
    service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        range=title_range,
        valueInputOption="USER_ENTERED",
        body=title_body
    ).execute()

    # Update descriptions in column C
    description_range = f"{sheet_name}!C2:C"
    description_body = {'values': [[description] for description in descriptions]}
    service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        range=description_range,
        valueInputOption="USER_ENTERED",
        body=description_body
    ).execute()

    # Update scores in column H
    score_range = f"{sheet_name}!H2:H"
    score_body = {'values': [[score] for score in scores]}
    service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        range=score_range,
        valueInputOption="USER_ENTERED",
        body=score_body
    ).execute()

# Setup the ChromeDriver
chrome_driver_path = 'C:\\Users\\djyan\\Downloads\\chromedriver-win32\\chromedriver-win32\\chromedriver.exe'
service = Service(executable_path=chrome_driver_path)
driver = webdriver.Chrome(service=service)

# Setup Google Sheets API
creds = Credentials.from_authorized_user_file('C:\\Users\\djyan\\Downloads\\client_secret_204709315294-t82p97ghvt54els62217rlgp3fj1d6jo.apps.googleusercontent.com (1).json', scopes=['https://www.googleapis.com/auth/spreadsheets'])
service = build('sheets', 'v4', credentials=creds)

# Your Google Sheets Spreadsheet ID
spreadsheet_id = '1tdwufrfmL0Bc2lnehOEopSo9T9w717LvbsjNhT8xqrE'

try:
    driver.get("https://clip.opus.pro/clip/P0123005tWpp")
    wait = WebDriverWait(driver, 20)
    wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, ".MuiTypography-root.MuiTypography-base.css-1wc18uy")))

    titles = driver.find_elements(By.CSS_SELECTOR, ".MuiTypography-root.MuiTypography-xl2.css-1jfjns0")
    descriptions = driver.find_elements(By.CSS_SELECTOR, ".MuiTypography-root.MuiTypography-base.css-1wc18uy")
    scores = driver.find_elements(By.CSS_SELECTOR, ".MuiTypography-root.MuiTypography-xl3.css-tz3tqc")

    specific_sheet_name = "Copy of Tim Draper"

    titles_list = []
    descriptions_list = []
    scores_list = []
    for title, description, score in zip(titles, descriptions, scores):
        score_value = int(score.text)
        if score_value > 80:
            titles_list.append(title.text)
            descriptions_list.append(description.text)
            scores_list.append(str(score_value))

    # Sort the data by score in descending order
    combined = list(zip(titles_list, descriptions_list, scores_list))
    combined.sort(key=lambda x: int(x[2]), reverse=True)  # Sort by scores
    titles_list, descriptions_list, scores_list = zip(*combined)

    # Update the specific sheet with sorted data
    update_google_sheet(service, spreadsheet_id, specific_sheet_name, titles_list, descriptions_list, scores_list)

finally:
    driver.quit()